# pandas统计分析基础实训

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.2f}'.format)


In [ ]:
# 设置每列最大宽度（比如 100）
pd.set_option('display.max_colwidth', 100)

# 设置显示的最大列数（比如 20）
pd.set_option('display.max_columns', 20)

# 设置显示宽度（比如 1000）
pd.set_option('display.width', 1000)


## 实训1 读取并查看某地区房屋销售数据的基本信息

“居”是民生的重要组成部分，也是百姓幸福生活的重要保障。为了增进民生福祉，提高人民生活品质，对房地产市场进行精准调控决策，现需分析和统计某地区房屋销售数据。该地区房屋销售数据主要存放了房屋售出时间、地区邮编、房屋价格、房屋类型和配套房间数5个特征，部分数据如表3-24 所示，其中房屋类型有普通住宅（house）和单身公寓（unit）两种。探索数据的基本信息，通过索引操作查询到房屋类型为单身公寓的数据，同时观察效据的整体分布并发现数据间的关联。注意，地区邮编特征已完成脱敏处理，因此只存在4位数。

实现思路及步骤
- 使用read_csv函数读取某地区房屋销售数据.csv文件
- 使用ndim、shape、columns属性分别查看数据的维度、形状，以及所有特征名称
- 使用iloc()方法、loc()方法对房屋类型为单身公寓的数据进行索引操作

In [ ]:
house = pd.read_csv("pd_data/某地区房屋销售数据.csv",encoding='gbk')

In [ ]:
print("数据维度（ndim）:", house.ndim) 
print("数据形状（shape）:", house.shape)        
print("特征名称（columns）:", house.columns)    

In [ ]:
studio_loc = house.loc[house["房屋类型"] == "unit"]

In [ ]:
studio_loc

In [ ]:
studio_iloc = house.iloc[(house["房屋类型"] == "unit").values,:]

In [ ]:
studio_iloc

## 实训2 提取房屋售出时间信息并描述房屋价格信息

基于实训1的数据，在房屋售出时间特征中存在时间数据，提取时间数据内存在的有用信息，如将“2010/1/40:00”转换成“2010-1-4”的形式。对时间信息的提取一方面可以加深房产销售经理对数据的理解，另一方面能够去除无意义的时间信息。此外，还可通过描述性统计分析该地区房屋的平均价格、价格区间、价格众数等，便于进一步获取该地区房屋价格信息。

实现思路及步骤
- 使用to_datetime函数转换房屋出售时间字符串
- 使用mean、max、min、mode 函数分别计算该地区房屋价格的均值、最大值、最小值和众数；
- 使用quantile函数计算该地区房屋价格的分位数。
- 使用describe方法计算房屋价格数据的非空值数目、均值等统计量。

In [ ]:
house["房屋出售时间"] = pd.to_datetime(house["房屋出售时间"], errors="coerce")

In [ ]:
mean_price = house["房屋价格"].mean()
max_price = house["房屋价格"].max()
min_price = house["房屋价格"].min()
mode_price = house["房屋价格"].mode()[0]

In [ ]:
print("均值:", mean_price)
print("最大值:", max_price)
print("最小值:", min_price)
print("众数:", mode_price)

In [ ]:
q25 = house["房屋价格"].quantile(0.25)  # 第一四分位数
q50 = house["房屋价格"].quantile(0.50)  # 中位数
q75 = house["房屋价格"].quantile(0.75)  # 第三四分位数

In [ ]:

print("25% 分位数:", q25)
print("50% 分位数（中位数）:", q50)
print("75% 分位数:", q75)

In [ ]:
desc_stats = house["房屋价格"].describe()
print(desc_stats)

## 实训3 使用分组聚合方法分析房屋销售情况

为了解买房者购买房屋的类型喜好，需要根据房屋所在的地理位置进行分组聚合，然后进行组内和组间分析，从而为买房者提供更好的服务。基于实训1的数据，提取地区邮编特征中数据的前两位，如提取“2615”中的“26”，并生成new_postcode 特征存储提取的内容，其目的是便于统计不同地区房屋价格以及房屋性价比。最后根据newpostcode特征对数据进行分组操作，从而获取不同地区的房屋价格信息并进行比较。

实现思路及步骤
- 使用apply()方法生成new_postcode特征。
- 使用agg()方法和count函数计算出每个地区的房屋售出总数。
- 使用groupby()方法对房屋类型propertyType 进行分组，并对新地区邮编new_postcode进行分组后赋值给新的数据框housesale1。
- 使用transform()聚合方法和mean函数计算housesale1中房屋价格的均值。

In [ ]:
house["new_postcode"] = house["地区邮编"].apply(lambda x: str(x)[:2])

In [ ]:
sale_count = house.groupby("new_postcode").agg({"房屋价格": "count"}).rename(columns={"房屋价格": "售出总数"})
print(sale_count)

In [ ]:
housesale1 = house.groupby(["房屋类型", "new_postcode"]).apply(lambda x: x).reset_index(drop=True)
print(housesale1)

In [ ]:
housesale1["均价"] = housesale1.groupby(["房屋类型", "new_postcode"])["房屋价格"].transform("mean")
print(housesale1)

## 实训4 分析房屋地区、配套房间数和房屋价格的关系

通过实训 1~3中对数据进行描述性统计、对时间数据进行信息提取和分组聚合操作，已获得了相当多的信息。基于实训1的数据，如果需要获取更多的房屋价格信息，那么可以使用透视表和交叉表来实现。例如，比较不同地区和不同配套房间数的房屋价格、分析不同地区哪种类型的房屋价格最贵、比较不同类型房屋和不同配套房间数的房屋的价格和分析配有多少房间数的房屋最畅销等

实现思路及步骤
- 使用pivot_table函数创建数据透视表。
- 使用crosstab函数创建数据交叉表。

In [ ]:
pivot = pd.pivot_table(
    house,
    values="房屋价格",
    index="new_postcode",
    columns="房屋类型",
    aggfunc="mean"
)
print(pivot)

In [ ]:
cross = pd.crosstab(house["new_postcode"], house["房屋类型"])
print(cross)